In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.gene import get_coding_genetic_target_len_d, get_intergenic_len_d
from util.genome import get_feature_hit_set
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
assoc_muts_df = pd.read_pickle("./data/6_df.pkl")
display(assoc_muts_df.shape, assoc_muts_df.head())

(148, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
630,66,T247M (ACG→ATG),yafD,SNP,231861,NC_000913,C→T,1,GYD,280,0,0.72,1,True,"(231861, 231861)",{ECK120001594},"[{'name': 'yafD', 'RegulonDB ID': 'ECK12000159...",False,False,{},{},{},{},{},True,"[{'name': 'yafD', 'RegulonDB ID': 'ECK12000159...",{'ECK120001594': ['ECK120001594']},"[{'name': 'yafDE', 'RegulonDB ID': 'ECK1200291...",{'ECK120029169': ['ECK120001594']},[],{},[],{},[{'name': 'General function prediction only'}],{'General function prediction only': ['ECK1200...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
631,126,R665S (CGT→AGT),cyaA,SNP,3993145,NC_000913,C→A,1,GYD,280,0,1.00,1,True,"(3993145, 3993145)",{ECK120000166},"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12000016...",False,False,{},{},{},{},{},True,"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12000016...",{'ECK120000166': ['ECK120000166']},"[{'name': 'cyaA', 'RegulonDB ID': 'ECK12001483...",{'ECK120014835': ['ECK120000166']},"[{'name': 'CRP', 'significantly associated con...",{'CRP': ['cyaA']},[{'name': 'Purine metabolism'}],{'Purine metabolism': ['ECK120014835']},[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
632,94,S59Y (TCC→TAC),glpK,SNP,4117047,NC_000913,G→T,1,GYD,280,0,1.00,1,True,"(4117047, 4117047)",{ECK120000391},"[{'name': 'glpK', 'RegulonDB ID': 'ECK12000039...",False,False,{},{},{},{},{},True,"[{'name': 'glpK', 'RegulonDB ID': 'ECK12000039...",{'ECK120000391': ['ECK120000391']},"[{'name': 'glpFKX', 'RegulonDB ID': 'ECK120014...",{'ECK120014723': ['ECK120000391']},"[{'name': 'GlpR', 'significantly associated co...","{'GlpR': ['glpFKX'], 'CRP': ['glpFKX']}",[{'name': 'Glycerolipid metabolism'}],{'Glycerolipid metabolism': ['ECK120014723']},[{'name': 'Energy production and conversion'}],{'Energy production and conversion': ['ECK1200...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
633,72,T593P (ACC→CCC),metL,SNP,4131611,NC_000913,A→C,1,GYD,280,0,0.67,1,True,"(4131611, 4131611)",{ECK120000583},"[{'name': 'metL', 'RegulonDB ID': 'ECK12000058...",False,False,{},{},{},{},{},True,"[{'name': 'metL', 'RegulonDB ID': 'ECK12000058...",{'ECK120000583': ['ECK120000583']},"[{'name': 'metBL', 'RegulonDB ID': 'ECK1200150...",{'ECK120015057': ['ECK120000583']},"[{'name': 'MetJ', 'significantly associated co...","{'MetJ': ['metBL'], 'PhoP': ['metBL'], 'HypT':...","[{'name': 'Glycine, serine and threonine metab...","{'Glycine, serine and threonine metabolism': [...",[{'name': 'Amino acid transport and metabolism'}],{'Amino acid transport and metabolism': ['ECK1...,37 celsius,glycerol(2.5),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
634,107,E641K (GAA→AAA),rpoB,SNP,4183165,NC_000913,G→A,1,GYD,280,0,1.00,1,True,"(4183165, 4183165)",{ECK120000885},"[{'name': 'rpoB', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoB', 'RegulonDB ID': 'ECK12000088...",{'ECK120000885': ['ECK120000885']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000885']},[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},37 celsius,glycerol(2.5)

# Build genetic feature links dictionary for mutflow

In [3]:
# # Now in NB 2.2.1. Remove cell becuause obsolete
# def get_genetic_links_d(mut_row):
#     gen_trgt_d = dict()
#     feat_df = pd.DataFrame()
#     for feat_d in mut_row["genomic features"]:
#         feat_df = feat_df.append(feat_d, ignore_index=True)
#     for gen_trgt in mut_row["genetic features"]:
#         gen_trgt_d[gen_trgt["RegulonDB ID"]] = list(get_feature_hit_set(gen_trgt["range"], feat_df, "range", "RegulonDB ID"))
#     return gen_trgt_d


# assoc_muts_df["genetic feature links"] = assoc_muts_df.apply(get_genetic_links_d, axis=1)
# assoc_muts_df.head()

# Significance study (level specific)

In [4]:
from util.feature import get_feat_mut_cnt_df_from_links
comp_mut_df = get_feat_mut_cnt_df_from_links(assoc_muts_df, "genetic features", "genetic feature links")
comp_mut_df

,length,observed mutation count,name
ECK120001594,801,1,yafD
ECK120000166,2547,21,cyaA
ECK120000391,1509,30,glpK
ECK120000583,2433,2,metL
ECK120000885,4029,5,rpoB
...,...,...,...
ECK120003261,1032,1,ynaI
ECK120003260/ECK120003261,50,0,mppA/ynaI
ECK120003259/ECK120003260,336,4,pgrR/mppA
ECK120003261/ECK120003262,153,4,ynaI/insH-4


In [5]:
for _, r in assoc_muts_df.iterrows():
    for feat_d in r["genetic features"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["RegulonDB ID"]]["observed mutation count"]

In [6]:
mutated_seq_len = np.sum(comp_mut_df["length"])

TOTAL_GENOME_LEN = 4641652
genome_remainder_len = TOTAL_GENOME_LEN - mutated_seq_len
df = pd.DataFrame([[0, genome_remainder_len]], columns=["observed mutation count", "length"], index=["genome remainder"])
comp_mut_df = comp_mut_df.append(df, sort=False)
total_seq_len = TOTAL_GENOME_LEN


comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

,length,observed mutation count,name,proportion
ECK120001594,801,1,yafD,0.000173
ECK120000166,2547,21,cyaA,0.000549
ECK120000391,1509,30,glpK,0.000325
ECK120000583,2433,2,metL,0.000524
ECK120000885,4029,5,rpoB,0.000868


In [7]:
from collections import Counter
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=num_muts,
        replace=True,
        p=proportion_l
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,length,observed mutation count,name,proportion,equal or larger count,p value,significant
ECK120001594,801,1,yafD,0.000173,306,0.0306,True
ECK120000166,2547,21,cyaA,0.000549,0,0.0000,True
ECK120000391,1509,30,glpK,0.000325,0,0.0000,True
ECK120000583,2433,2,metL,0.000524,43,0.0043,True
ECK120000885,4029,5,rpoB,0.000868,0,0.0000,True


In [8]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,length,observed mutation count,name,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
ECK120001594,801,1,yafD,0.000173,306,0.0306,True,1.0000,False
ECK120000166,2547,21,cyaA,0.000549,0,0.0000,True,0.0000,True
ECK120000391,1509,30,glpK,0.000325,0,0.0000,True,0.0000,True
ECK120000583,2433,2,metL,0.000524,43,0.0043,True,0.3225,False
ECK120000885,4029,5,rpoB,0.000868,0,0.0000,True,0.0000,True


In [9]:
# # not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [10]:
for i, r in assoc_muts_df.iterrows():
    for d in r["genetic features"]:
        if d["RegulonDB ID"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["RegulonDB ID"]]["final significance"]

# Outputting dataframe for the next step in the pipeline

In [11]:
assert(len(assoc_muts_df.exp.unique())==1)
exp = assoc_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_gen_feat_signif.csv")

In [12]:
assoc_muts_df.to_pickle("./data/7_df.pkl")

In [13]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_ID(RegulonDB_ID, json):
#     name = ""
#     is_signif = False
#     for d in json:
#         if d["RegulonDB ID"] == RegulonDB_ID:
#             name = d["name"]
#             is_signif = d["significant"]
#             break
#     return (name, is_signif)


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["genetic feature links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["genetic features"])
#         t = str(t)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 genetic mutated feature count")
# ax.get_legend().remove()